In [3]:
import networkx as nx
import math
import random
import numpy as np
import copy
from datetime import datetime
from matplotlib import pyplot as plt
import sys
import import_ipynb
import GraphGenerator as gg

importing Jupyter notebook from GraphGenerator.ipynb


In [4]:
# gg.build_graph()

In [5]:
def move_obstacle(g, node, start_pos, goal_pos, obstacles, stuck_num):
    if node not in obstacles:
        return False, None, None
    ## segFault
    if stuck_num == 50:
        return False, node, None
    
    possible_moves = sorted([node for node in g[node] if node != goal_pos and node != start_pos])
    # #recursion
    if len(possible_moves) == 0:
        return False, node, None
    
    chosen_move = random.sample(possible_moves, k=1)[0]
    if(chosen_move in obstacles):
        #recursion
        return move_obstacle(g, chosen_move, start_pos, goal_pos, obstacles, stuck_num+1)
        
    
    obstacles.remove(node)
    obstacles.append(chosen_move)

    # print(f'Obstacle moves: {node} -> {chosen_move}')
    return True, chosen_move, node

In [6]:
def BFS(g, start_pos, goal_pos, obstacles):
    upcoming_nodes = [start_pos]
    parents = {}
    visited = set()
    
    while (len(upcoming_nodes) != 0):
        u = upcoming_nodes.pop(0)
        visited.add(u)
        if (u == goal_pos):
            return parents
            
        neighbors = [v for v in g[u] if v not in obstacles and v not in visited]
        if len(neighbors) == 0:
            continue
            
        for neighbor in neighbors:
            upcoming_nodes.append(neighbor)
            parents[neighbor] = u

    return None

In [7]:
def find_path(g, start_pos, goal_pos, obstacles, num_iterations):
    min_length = float('inf')
    min_iteration = float('inf')
    obstacles_start = copy.deepcopy(obstacles)
    start_time = datetime.now()

    best_moves = []
    length_per_iteration = []
    obstacle_moves = []
    best_obstacle_moves = []
    break_indicator = 0

    for iteration in range(num_iterations):
        obstacle_moves.clear()
        iter_start_time = datetime.now()
        while (True):
            path = []
            parents = BFS(g, start_pos, goal_pos, obstacles)
            current_node = goal_pos
            if parents is not None:
                while (current_node != start_pos):
                    path.append(current_node)
                    current_node = parents[current_node]
                path.append(start_pos)
                path.reverse()
                break
            else:
                num_obstacles_to_move = max(len(obstacles)//4, 1)
                obstacles_to_move = random.sample(obstacles, k=num_obstacles_to_move)
                for obstacle in obstacles_to_move:
                    obstacle_moved, moved_to, what_was_moved = move_obstacle(g, obstacle, start_pos, goal_pos, obstacles, stuck_num=0)
                    if (obstacle_moved):
                        obstacle_moves.append(f'{what_was_moved}->{moved_to}')
                        
            if (datetime.now() - iter_start_time).total_seconds() > 0.5:
                print('Preskocena iteracija')
                break_indicator = 1
                break
        if break_indicator:
            break_indicator = 0
            continue
        num_of_steps = len(path)-1 + len(obstacle_moves)
        length_per_iteration.append(num_of_steps)
        # print(f'Path length for iteration {iteration+1}: {num_of_steps}')
        if num_of_steps < min_length:
            min_length = num_of_steps
            min_iteration = iteration + 1
            best_moves = copy.deepcopy(path)
            best_obstacle_moves = copy.deepcopy(obstacle_moves)
        
        obstacles = copy.deepcopy(obstacles_start)

    ###
    end_time = datetime.now()
    total_time = (end_time - start_time).total_seconds()
    
    solution_metrics = {
        "best_value": min_length,
        "time": total_time,
        "obstacles_moved": obstacle_moves,
        "best_path": best_moves,
        "solution_values": length_per_iteration
    }
    
    print(f'The minimum length of {solution_metrics["best_value"]} was found in iteration {min_iteration}')
    print(f'Time it took to finish the search: {solution_metrics["time"]}')
    print(f'Obstacles moved: {solution_metrics["obstacles_moved"]}')
    print(f'Best path of all iterations: {solution_metrics["best_path"]}')
    
#     for i in range(len(best_obstacle_moves)):
#         print(best_obstacle_moves[i])

    return solution_metrics

In [8]:
# find_path(G, robot_pos, goal_pos, obstacles, num_iterations=100)